In [1]:
# Start: Import Libraries
# ---------------------------
import os
import pandas as pd
import datetime
from openpyxl import load_workbook
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from reportlab.lib import colors
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet

In [2]:
# Part 1: Data Processing Functions
# ---------------------------
def process_excel_file(file_path):
    """
    Reads an Excel file with three sheets and splits each sheet into datasets.
    
    For each sheet:
      - Starting from the 3rd row (index 2), every two consecutive rows form a dataset.
      - For each dataset:
          - The data (to be used in return calculations) are taken from the 3rd column onward.
          - The first two columns of the first row in the pair are extracted as names.
    
    Note: The Excel file is assumed to have all data in text format.
    
    Returns:
        A dictionary mapping sheet names to a list of dataset dictionaries. Each dataset dict has:
          - 'name1': Insurer name.
          - 'name2': Fund Name.
          - 'data': A 2-row DataFrame where:
                    - Row 0 contains date strings in dd/mm/yyyy format.
                    - Row 1 contains numeric values as text.
    """
    xls = pd.ExcelFile(file_path)
    sheets_data = {}
    
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(file_path, sheet_name=sheet_name, header=None, dtype=str)
        datasets = []
        
        # Starting from 3rd row (index 2), take pairs of rows.
        for i in range(2, len(df), 2):
            if i + 1 < len(df):
                name1 = df.iloc[i, 0]
                name2 = df.iloc[i, 1]
                data = df.iloc[i:i+2, 2:]
                datasets.append({
                    'name1': name1,
                    'name2': name2,
                    'data': data
                })
            else:
                print(f"Warning: Row {i+1} in sheet '{sheet_name}' does not have a pair; skipping.")
        
        sheets_data[sheet_name] = datasets
    
    return sheets_data

# def get_closest_data_point(dates, values, target_date):
#     """
#     Given lists of dates (as datetime objects) and corresponding values, find the data point
#     (i.e. date and value) where the date is the largest date that is less than or equal to target_date.
    
#     Returns:
#         (closest_date, corresponding_value, index)
    
#     Raises:
#         ValueError if no date in the list is <= target_date.
#     """
    
#     candidates = [(i, dt) for i, dt in enumerate(dates) if dt <= target_date]
#     print(len(dates), len(candidates))
#     if not candidates:
#         raise ValueError("No date found that is less than or equal to the target date.")
#     candidate_idx, candidate_date = max(candidates, key=lambda x: x[1])
#     print(max(candidates, key=lambda x: x[1]))
#     candidate_value = values[candidate_idx]
#     print(candidate_date, candidate_value, candidate_idx)
#     return candidate_date, candidate_value, candidate_idx

def get_closest_data_point(dates, values, target_date):
    """
    Given lists of dates (as datetime objects) and corresponding values,
    this function sorts the data by date and then finds the data point (i.e. date and value)
    where the date is the largest that is less than or equal to target_date.
    
    Returns:
        (closest_date, corresponding_value, original_index)
    
    Raises:
        ValueError if no date in the list is <= target_date.
    """
    # Create a list of tuples: (original_index, date, value)
    paired = list(zip(range(len(dates)), dates, values))
    # Sort the list by date (ascending)
    paired_sorted = sorted(paired, key=lambda x: x[1])
    
    # Filter the sorted pairs for those with date <= target_date
    sorted_candidates = [ (i, dt, val) for i, dt, val in paired_sorted if dt <= target_date ]
    
    if not sorted_candidates:
        raise ValueError("No date found that is less than or equal to the target date.")
    
    # The candidate we want is the last one in the sorted order.
    candidate_idx, candidate_date, candidate_value = sorted_candidates[-1]
    
    # For debugging, if you want to see the position in the sorted candidates list:
    # candidate_sorted_position = len(sorted_candidates) - 1
    # print("Total dates:", len(dates), "Candidates in sorted order:", len(sorted_candidates),
    #       "Candidate sorted position:", candidate_sorted_position)
    
    return candidate_date, candidate_value, candidate_idx


def calculate_returns(data):
    """
    Given a 2-row DataFrame (row 0: dates as dd/mm/yyyy strings, row 1: numeric values as text),
    compute various return metrics using the formula:
    
        (current value - previous value) / previous value * (365 * 100) / days_difference
    
    The computed return is rounded to two decimals.
    
    Returns a dictionary with keys:
      'daily', 'weekly', 'monthly', '3-monthly', '6-monthly', 'FYTD', and 'latest_NAV'.
    Note: The MTD return calculation has been commented out.
    """
    date_strs = data.iloc[0].tolist()
    value_strs = data.iloc[1].tolist()
    
    parsed_dates = []
    parsed_values = []
    
    # Process each column; skip if date is missing or "nan"
    for dt, val in zip(date_strs, value_strs):
        dt_str = str(dt).strip() if dt is not None else ""
        if dt_str == "" or dt_str.lower() == "nan":
            continue
        try:
            parsed_date = datetime.strptime(dt_str, "%d/%m/%Y")
        except Exception as e:
            raise ValueError(f"Error parsing date '{dt_str}': {e}")
        try:
            num_val = float(val)
        except Exception as e:
            raise ValueError(f"Error converting value '{val}' to float: {e}")
        parsed_dates.append(parsed_date)
        parsed_values.append(num_val)

    if len(parsed_dates) < 2:
        raise ValueError("Not enough valid data points to compute returns.")
    
    # The current (latest) data point.
    current_date = parsed_dates[-1]
    current_value = parsed_values[-1]
    
    results = {}
    
    def compute_return(prev_value, days_diff):
        if prev_value == 0 or days_diff == 0:
            return None
        ret = ((current_value - prev_value) / prev_value) * (365 * 100) / days_diff
        return round(ret, 2)
    
    # Daily return: use the immediate preceding valid data point.
    prev_date_daily = parsed_dates[-2]
    prev_value_daily = parsed_values[-2]
    d_1 = (current_date - prev_date_daily).days
    daily_return = compute_return(prev_value_daily, d_1)
    results['daily'] = {"d_1": d_1, "return": daily_return}
    
    # Weekly return: target = current_date - 7 days.
    required_weekly = current_date - timedelta(days=7)
    try:
        prev_date_weekly, prev_value_weekly, _ = get_closest_data_point(parsed_dates, parsed_values, required_weekly)
        d_7 = (current_date - prev_date_weekly).days
        weekly_return = compute_return(prev_value_weekly, d_7)
    except ValueError:
        d_7 = None
        weekly_return = None
    results['weekly'] = {"d_7": d_7, "return": weekly_return}
    
    # 1-month return: target = current_date - 1 month.
    required_monthly = current_date - relativedelta(months=1)
    try:
        prev_date_monthly, prev_value_monthly, _ = get_closest_data_point(parsed_dates, parsed_values, required_monthly)
        d_m = (current_date - prev_date_monthly).days
        monthly_return = compute_return(prev_value_monthly, d_m)
    except ValueError:
        d_m = None
        monthly_return = None
    results['monthly'] = {"d_m": d_m, "return": monthly_return}
    
    # 3-month return: target = current_date - 3 months.
    required_3m = current_date - relativedelta(months=3)
    try:
        prev_date_3m, prev_value_3m, _ = get_closest_data_point(parsed_dates, parsed_values, required_3m)
        d_3m = (current_date - prev_date_3m).days
        ret_3m = compute_return(prev_value_3m, d_3m)
    except ValueError:
        d_3m = None
        ret_3m = None
    results['3-monthly'] = {"d_3m": d_3m, "return": ret_3m}
    
    # 6-month return: target = current_date - 6 months.
    required_6m = current_date - relativedelta(months=6)
    try:
        prev_date_6m, prev_value_6m, _ = get_closest_data_point(parsed_dates, parsed_values, required_6m)
        d_6m = (current_date - prev_date_6m).days
        ret_6m = compute_return(prev_value_6m, d_6m)
    except ValueError:
        d_6m = None
        ret_6m = None
    results['6-monthly'] = {"d_6m": d_6m, "return": ret_6m}
    
    # FYTD return: fixed target date, here 01/04/2024.
    required_fytd = datetime.strptime("01/04/2025", "%d/%m/%Y")
    try:
        prev_date_fytd, prev_value_fytd, _ = get_closest_data_point(parsed_dates, parsed_values, required_fytd)
        d_y = (current_date - prev_date_fytd).days
        fytd_return = compute_return(prev_value_fytd, d_y)
    except ValueError:
        d_y = None
        fytd_return = None
    results['FYTD'] = {"d_y": d_y, "return": fytd_return}
    
    # # MTD return calculation is commented out.
    # required_mtd = current_date.replace(day=1)
    # try:
    #     prev_date_mtd, prev_value_mtd, _ = get_closest_data_point(parsed_dates, parsed_values, required_mtd)
    #     d_mtd = (current_date - prev_date_mtd).days
    #     mtd_return = compute_return(prev_value_mtd, d_mtd)
    # except ValueError:
    #     d_mtd = None
    #     mtd_return = None
    # results['MTD'] = {"d_mtd": d_mtd, "return": mtd_return}
    
    # Store the current (latest) value (Latest NAV).
    results['latest_NAV'] = current_value
    # print(results)
    return results

In [3]:
# Part 2: PDF Report Generation
# ---------------------------
def create_pdf_report(sheets_datasets, output_pdf, date):
    """
    Creates a PDF report with a bold title and one table per sheet.
    
    Each table has 9 columns:
      "Insurer", "Fund Name", "Latest NAV", "Daily return", "Weekly return", 
      "1-month return", "3-month return", "6-month return", "FYTD return".
      
    The table rows are sorted by Daily return (largest to smallest).
    At the end of the PDF, two notes are added:
      *Axis Maxlife NAV values are rounded off to 2 digits after decimal
      **All returns are calculated as percentage(%)
    """
    sheet_tables = {}
    
    # Build table data for each sheet.
    for sheet_name, datasets in sheets_datasets.items():
        rows = []
        for dataset in datasets:
            insurer = dataset['name1']
            fund_name = dataset['name2']
            if insurer not in exclude_fund:
                print(insurer, fund_name)
                try:
                    ret = calculate_returns(dataset['data'])
                    daily = ret.get('daily', {}).get('return', None)
                    weekly = ret.get('weekly', {}).get('return', None)
                    monthly = ret.get('monthly', {}).get('return', None)
                    three_monthly = ret.get('3-monthly', {}).get('return', None)
                    six_monthly = ret.get('6-monthly', {}).get('return', None)
                    fytd = ret.get('FYTD', {}).get('return', None)
                    latest_nav = ret.get('latest_NAV', None)
                except Exception as e:
                    print(e)
                    continue  # Skip dataset if returns cannot be computed.
                
                # New row: Insert Latest NAV between Fund Name and Daily return.
                rows.append([insurer, fund_name, latest_nav, daily, weekly, monthly, three_monthly, six_monthly, fytd])
    
        # Sort rows by Daily return (index 3) descending.
        rows_sorted = sorted(rows, key=lambda x: x[3] if x[3] is not None else -float('inf'), reverse=True)
        sheet_tables[sheet_name] = rows_sorted

    # Setup the PDF document.
    doc = SimpleDocTemplate(output_pdf, pagesize=A4,
                            leftMargin=36, rightMargin=36, topMargin=36, bottomMargin=36)
    elements = []
    styles = getSampleStyleSheet()
    
    # Title: Bold title with current date.
    title_text = "Life Insurance Fund Performance Report for NAV dated " + date
    title_paragraph = Paragraph("<b>" + title_text + "</b>", styles['Title'])
    elements.append(title_paragraph)
    elements.append(Spacer(1, 12))
    
    # Define headers with updated names.
    headers = ["Insurer", "Fund Name", "Latest NAV", "Daily return", "Weekly return", 
               "1-month return", "3-month return", "6-month return", "FYTD return"]
    
    # Define fixed column widths (adjust as needed).
    col_widths = [60, 80, 40, 50, 50, 50, 50, 50, 40]
    
    # For each sheet, add a heading and a table.
    for sheet, rows in sheet_tables.items():
        sheet_title = Paragraph(f"<b>{sheet}</b>", styles['Heading2'])
        elements.append(sheet_title)
        elements.append(Spacer(1, 6))
        
        table_data = [headers]
        table_data.extend(rows)
        
        table = Table(table_data, colWidths=col_widths, repeatRows=1)
        table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, -1), 6),
            ('BOTTOMPADDING', (0, 0), (-1, 0), 4),
            ('TOPPADDING', (0, 0), (-1, 0), 4),
            ('LEFTPADDING', (0, 0), (-1, -1), 2),
            ('RIGHTPADDING', (0, 0), (-1, -1), 2),
            ('GRID', (0, 0), (-1, -1), 0.5, colors.black),
        ]))
        
        elements.append(table)
        elements.append(Spacer(1, 12))
    
    # Add notes at the end of the PDF.
    # note_text = ("<para align='left'>"
    #              "*Axis Maxlife NAV values are rounded off to 2 digits after decimal<br/>"
    #              "*All returns are calculated as percentage(%)<br/>"
    #              # "*All Daily returns for this report are 2-day return"
    #              # "*Aditya Birla returns for this report is 4-day return"
    #              # "*ICICI Prudential returns are not included in today's report due to a technical difficulty"
    #              "</para>")
    note_paragraph = Paragraph(note_text, styles['Normal'])
    elements.append(note_paragraph)
    
    doc.build(elements)
    print("PDF report generated:", output_pdf)

#### First uncomment the fund you want to exclude, comment all others
#### If no fund is to be excluded, comment all the names

In [4]:
exclude_fund = [
    # "ICICI Prudential",
    # "SBI Life",
    # "Maxlife",
    # "Bajaj Allianz",
    # "LIC",
    # "HDFC",
    # "Aditya Birla",
    # "Tata AIA",
    # "PNB Metlife",
    # "Kotak Mahindra"
]

#### Text to write at the end of the report, uncomment and change lines as needed

In [5]:
note_text = ("<para align='left'>"
                 # "*Axis Maxlife NAV values are rounded off to 2 digits after decimal<br/>"
                 "*All returns are calculated as percentage(%)<br/>"
                 # "*All Daily returns for this report are 2-day return"
                 # "*Aditya Birla returns for this report is 4-day return"
                 # "*ICICI Prudential returns are not included in today's report due to a technical difficulty"
                 "</para>")

In [6]:
# Part 3: Main Usage
# ---------------------------
if __name__ == "__main__":
    excel_file_path = "D:/Innover/Advanced_funds.xlsx" # Update this path to your Excel file
    exclude_fund.append(None)
    sheets_datasets = process_excel_file(excel_file_path)
    
    date = (datetime.today() - timedelta(days=1)).strftime("%d-%b-%Y")
    output_pdf = f"D:/Innover/{date}_Insurance_Fund_Performance_Report.pdf"
    
    # output_pdf = f"D:/Innover/spare.pdf"
    create_pdf_report(sheets_datasets, output_pdf, date)

HDFC Income Fund
HDFC Bond Fund
HDFC Pension Income Fund
ICICI Prudential Income Fund
ICICI Prudential Life Secure Fund
Kotak Mahindra Kotak Dynamic Bond Fund
Maxlife Secure Fund
Aditya Birla Income Advantage Fund
Tata AIA Wholelife Income Fund
SBI Life Bond Fund
SBI Life Bond Pension Fund II
SBI Life Corporate Bond Fund
Bajaj Allianz Bond Fund
PNB Metlife Protector Fund II
LIC Nivesh Plus Bond Fund
ICICI Prudential Money Market Fund
SBI Life Money Market Pension II
ICICI Prudential Group Debt Fund II
Kotak Mahindra Kotak Group Bond Fund
PDF report generated: D:/Innover/09-May-2025_Insurance_Fund_Performance_Report.pdf
